In [1]:
from Params import DataParams, NetworkParams, TrainingParams, Params
from data_helper import XORDataset, getVariableLengths
from Model import Model, train_model

In [2]:
def generateDataParams(max_seq_len, min_seq_len=5):
    def num_samples_fn(seq_len):
        max_seqs = 100000
        num_seqs = seq_len ** 2
        return min(num_seqs, max_seqs)
    data_params_list = []
    for seq_len in range(min_seq_len, max_seq_len+1):
        num_samples = num_samples_fn(seq_len)
        data_params_list.append(DataParams(num_samples, seq_len, False))
        data_params_list.append(DataParams(num_samples, seq_len, True))
    return data_params_list

In [3]:
max_seq_len = 50
data_params_list = generateDataParams(max_seq_len)
network_params = NetworkParams()
training_params = TrainingParams()

In [4]:
num_steps_varying_seq = dict()
num_steps_fixed_seq = dict()
for data_params in data_params_list:
    params = Params(data_params, network_params, training_params)
    model = Model(network_params)
    num_steps = train_model(model, params)
    if data_params.is_seq_len_varying:
        num_steps_varying_seq[data_params.max_seq_len] = num_steps
    else:
        num_steps_fixed_seq[data_params.max_seq_len] = num_steps


Seq len:5, Varying seq len:False
step 2450, loss 0.0915, accuracy 1.0000, test accuracy 0.9827

Seq len:5, Varying seq len:True
step 7700, loss 0.0102, accuracy 1.0000, test accuracy 0.9816

Seq len:6, Varying seq len:False
step 2100, loss 0.2344, accuracy 1.0000, test accuracy 0.9952

Seq len:6, Varying seq len:True
step 2550, loss 0.0890, accuracy 1.0000, test accuracy 0.9811

Seq len:7, Varying seq len:False
step 3450, loss 0.2767, accuracy 1.0000, test accuracy 0.9818

Seq len:7, Varying seq len:True
step 2100, loss 0.2930, accuracy 1.0000, test accuracy 0.9865

Seq len:8, Varying seq len:False
step 2250, loss 0.1429, accuracy 0.9922, test accuracy 0.9810

Seq len:8, Varying seq len:True
step 5400, loss 0.1575, accuracy 1.0000, test accuracy 0.9837

Seq len:9, Varying seq len:False
step 2350, loss 0.0896, accuracy 1.0000, test accuracy 0.9831

Seq len:9, Varying seq len:True
step 2800, loss 0.1987, accuracy 1.0000, test accuracy 0.9836

Seq len:10, Varying seq len:False
step 4300,